In [10]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
path="https://github.com/invinst/chicago-police-data/raw/master/Previous_Format/complaints-cpd-2016-nov/merged.csv"

In [3]:
#getting the original data from github (CPD complaints November 2016)
df0=pd.read_csv(path, low_memory=False)
columns=df0.columns.values.tolist()
print(columns)

['Complaint_Number', 'Beat', 'Location_Code', 'Address', 'Street', 'Apartment', 'City_State_Zipcode', 'Incident_Datetime', 'Complaint_Date', 'Closed_Date', 'Full_Address', 'Investigator_Name', 'Investigator_Current_Assignment', 'Investigator_Rank', 'Investigator_Star', 'Investigator_Date_Appointed', 'Accused_Name', 'Accused_Birth_Yr', 'Accused_Gender', 'Accused_Race_Code', 'Accused_Date_Appointed', 'Accused_Current_Unit', 'Accused_Current_Rank', 'Accused_Star', 'Accused_Complaint_Category', 'Accused_Finding', 'Accused_Recommended_Discipline', 'Accused_Final_Finding', 'Accused_Discipline', 'PO_Witness_Name', 'PO_Witness_Gender', 'PO_Witness_Race', 'PO_Witness_Star', 'PO_Witness_Birth_Year', 'PO_Witness_Date_Appointed', 'Victim_Gender', 'Victim_Age', 'Victim_Race_Desc', 'Complainant_Gender', 'Complainant_Age', 'Complainant_Race_Desc']


In [4]:

df1=df0[pd.isnull(df0.Accused_Name)==False] #extract all complaints for which we have an officer name attached to
df2=df0[pd.isnull(df0.Accused_Name)==True] #all complaint records with no officer names

In [5]:
print('Number of records with officer names......',len(df1),' records')
print('Number of records without officer names...',len(df2),'  records')

Number of records with officer names...... 133601  records
Number of records without officer names... 71486   records


In [12]:
unitsize=df1.groupby(['Accused_Current_Unit']).size()

In [23]:
unitsize.sort_values()

Accused_Current_Unit
74        1
73        1
156       1
159       1
162       1
611       1
602       1
147       1
411       1
PA2       2
154       2
157       2
622       2
612       2
176       2
91        2
139       3
720       3
75        3
413       4
71        5
412       5
119       6
53        7
72        7
545       8
216       8
112       9
170       9
130      10
       ... 
211    1611
212    1625
17     2220
14     2322
20     2340
24     2356
630    2368
15     2540
16     2808
620    3005
12     3009
610    3136
3      3200
25     3251
10     3302
193    3313
5      3506
11     3558
1      3652
19     3699
2      3726
7      3790
9      3838
6      3916
18     4218
22     4263
4      4351
376    5814
8      5876
189    6747
dtype: int64

In [48]:
#creation of the person database
onames=df1.Accused_Name.tolist()
onames=[(x.split(',')[0],x.split(',')[1]) for x in onames]
onames=[(x.strip(),y.strip()) for (x,y) in onames]
oinfo=list(zip(df1.Accused_Birth_Yr.tolist(),df1.Accused_Gender.tolist(),df1.Accused_Race_Code.tolist()))
ofcrs=list(zip(onames,oinfo))
ofcrs=[(l,f,b,g,r) for ((l,f),(b,g,r)) in ofcrs]

In [50]:
ofcrs_set=list(set(ofcrs))
potentialdups=len(ofcrs)-len(ofcrs_set)


print('Number of records matching on name, year of birth, gender, and race: ', potentialdups)
print('Number of unique officers (same name,birth year,gender, race): ', len(ofcrs_set))

Number of records matching on name, year of birth, gender, and race:  117589
Number of unique officers (same name,birth year,gender, race):  16012


In [153]:
df3=pd.DataFrame(ofcrs_set, columns=['LastName','FirstName','BirthYear','Gender','Race'])
#Race is notoriously unreliable in police dataset...
#Testing whether we can find matches based on last, first names, and birth year
df3['Duplicates_LN_FN_BY']=df3.duplicated(subset=['LastName','FirstName','BirthYear'], keep=False)
#Testing how often officers have the same LastName and FirstName
df3['Duplicates_LN_FN']=df3.duplicated(subset=['LastName','FirstName'], keep=False)
print('Number of duplicates based on LastName and FirstName: ',len(df3[df3.Duplicates_LN_FN==True]))
print('Number of duplicates based on LastName, FirstName, and BirthYear: ',len(df3[df3.Duplicates_LN_FN_BY==True]))


Number of duplicates based on LastName and FirstName:  949
Number of duplicates based on LastName, FirstName, and BirthYear:  10


In [154]:
#List of officers that match on First and Last Names
pd.options.display.max_rows = 1000
df3[df3.Duplicates_LN_FN==True].sort_values(by=['Duplicates_LN_FN','LastName','FirstName'], ascending=[False,True,True])

,LastName,FirstName,BirthYear,Gender,Race,Duplicates_LN_FN_BY,Duplicates_LN_FN
8001,ACEVEDO,EDWARD,1963,M,S,False,True
8110,ACEVEDO,EDWARD,1975,M,S,False,True
4435,ACOSTA,JESSE,1960,M,S,False,True
4946,ACOSTA,JESSE,1939,M,S,False,True
932,ADAMS,JAMES,1970,M,WHI,False,True
3629,ADAMS,JAMES,1944,M,WHI,False,True
7152,ADAMS,JAMES,1957,M,BLK,False,True
13963,ADE,JAMES,1985,M,WHI,False,True
14714,ADE,JAMES,1963,M,WHI,False,True
12211,ALCAZAR,VICTOR,1971,M,S,False,True


In [100]:
#List of officers that have the same LastName,FirstName, and BirthYear
df3[df3.Duplicates_LN_FN_BY==True].sort_values(by=['Duplicates_LN_FN_BY','LastName','BirthYear'], ascending=[False,True,True])

,LastName,FirstName,BirthYear,Gender,Race,Duplicates_LN_FN_BY,Duplicates_LN_FN
494,DOWNS,RICHARD,1953,M,BLK,True,True
10317,DOWNS,RICHARD,1953,M,WHI,True,True
3875,GRIFFIN,JOHN,1946,M,BLK,True,True
7512,GRIFFIN,JOHN,1946,M,WHI,True,True
1365,JOHNSON,MARK,1972,M,WHI,True,True
13498,JOHNSON,MARK,1972,M,BLK,True,True
3770,THOMAS,JOHN,1949,M,BLK,True,True
14520,THOMAS,JOHN,1949,M,WHI,True,True
7574,WILLIAMS,JOHN,1949,M,WHI,True,True
9254,WILLIAMS,JOHN,1949,M,BLK,True,True


In [155]:
#At this point, the best course of action is to create unique ids based on matches 
#on LastName,FirstName,BirthYear,Gender,and Race, but keep in mind those 10 cases above when we bring in more information

#Creation of unique id
df3=df3.sort_values(by=['Duplicates_LN_FN_BY','LastName','BirthYear']).reset_index()
df3ids=df3.index.values.tolist()
df3ids=[x+1 for x in df3ids]
df3['ofcr_id']=df3ids
#ids 16003 to 16012 correspond to the 10 records identified above
df3=df3.set_index('ofcr_id')
df3['ofcr_id']=df3ids

In [156]:
df3=df3.drop(['index','Duplicates_LN_FN_BY','Duplicates_LN_FN'], axis=1)


In [186]:
ofcr_info=df3.to_dict(orient='index')

In [157]:
#Merge the new ids with the incidents
def cleanname(row,last=True):
    if last==True:
        name=row.split(',')[0].strip()
    else:
        name=row.split(',')[1].strip()
    
    return name
df1['LastName']=df1.Accused_Name.apply(lambda x: cleanname(x,last=True))
df1['FirstName']=df1.Accused_Name.apply(lambda x: cleanname(x,last=False)) #don't worry about 'Setting with copy' warning
df1_wids=df1.merge(df3, how='left',left_on=['LastName','FirstName','Accused_Birth_Yr','Accused_Gender','Accused_Race_Code'],
                  right_on=['LastName','FirstName','BirthYear','Gender','Race'])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [158]:
#check if assignment was successful (should see 0 records)
df1_wids[pd.isnull(df1_wids.ofcr_id==True)]

,Complaint_Number,Beat,Location_Code,Address,Street,Apartment,City_State_Zipcode,Incident_Datetime,Complaint_Date,Closed_Date,...,Victim_Race_Desc,Complainant_Gender,Complainant_Age,Complainant_Race_Desc,LastName,FirstName,BirthYear,Gender,Race,ofcr_id


In [159]:
cplts=list(zip(df1_wids.Complaint_Number.tolist(),df1_wids.ofcr_id.tolist()))

In [161]:
#This attaches a list of unique complaint ids to officers. 
#NOTE: Officers can have multiple records within a single complaint number
ofcr_cplts={}
for (c,o) in cplts:
    if o not in ofcr_cplts.keys():
        ofcr_cplts[o]={'complaints':[]}
        if c not in ofcr_cplts[o]['complaints']:
            
            ofcr_cplts[o]['complaints'].append(c)
    else:
        if c not in ofcr_cplts[o]['complaints']:
            ofcr_cplts[o]['complaints'].append(c)

In [187]:
#Combines the list of complaint ids with the dictionary of unique officer info
for k,v in ofcr_info.items():
    ofcr_info[k].update(ofcr_cplts[k])

In [172]:
#creates a dictionary that links complaint events to the officers involved
cplt_ofcr={}
for k,v in ofcr_info.items():
    for c in v['complaints']:
        
        if c not in cplt_ofcr.keys():
            cplt_ofcr[c]={'officers':[]}
            if k not in cplt_ofcr[c]['officers']:
                cplt_ofcr[c]['officers'].append(k)
        else:
            if k not in cplt_ofcr[c]['officers']:
                cplt_ofcr[c]['officers'].append(k)
            
    

In [188]:
#Adds a field in the officer dictionary for the list of co-offenders they are involved with 
for k,v in ofcr_info.items():
    ofcr_info[k]['co-offenders']=[]
    for c in v['complaints']:
        co=[co for co in cplt_ofcr[c]['officers'] if co!=k]
        for c2 in co:
            ofcr_info[k]['co-offenders'].append(c2)
    

In [189]:
#Adds a field that counts the number of times an officer is involved with their co-offenders
from collections import Counter
for k,v in ofcr_info.items():
    offcounts=dict(Counter(ofcr_info[k]['co-offenders']))
    offcounts=[(k,v) for k,v in offcounts.items()]
    ofcr_info[k]['co-offenders_count']=offcounts
    
                   
    
        

In [203]:
print(columns)

['Complaint_Number', 'Beat', 'Location_Code', 'Address', 'Street', 'Apartment', 'City_State_Zipcode', 'Incident_Datetime', 'Complaint_Date', 'Closed_Date', 'Full_Address', 'Investigator_Name', 'Investigator_Current_Assignment', 'Investigator_Rank', 'Investigator_Star', 'Investigator_Date_Appointed', 'Accused_Name', 'Accused_Birth_Yr', 'Accused_Gender', 'Accused_Race_Code', 'Accused_Date_Appointed', 'Accused_Current_Unit', 'Accused_Current_Rank', 'Accused_Star', 'Accused_Complaint_Category', 'Accused_Finding', 'Accused_Recommended_Discipline', 'Accused_Final_Finding', 'Accused_Discipline', 'PO_Witness_Name', 'PO_Witness_Gender', 'PO_Witness_Race', 'PO_Witness_Star', 'PO_Witness_Birth_Year', 'PO_Witness_Date_Appointed', 'Victim_Gender', 'Victim_Age', 'Victim_Race_Desc', 'Complainant_Gender', 'Complainant_Age', 'Complainant_Race_Desc']


In [211]:
#Test whether different records with the same Complaint_Number have different attributes
df1_wids['Duplicates_CN_IDT']=df1_wids.duplicated(subset=['Complaint_Number', 'Incident_Datetime'], keep=False)
df1_wids['Duplicates_CN']=df1_wids.duplicated(subset=['Complaint_Number'], keep=False)
df1_wids['Duplicates_CN_B']=df1_wids.duplicated(subset=['Complaint_Number', 'Beat'], keep=False)
df1_wids['Duplicates_CN_LC']=df1_wids.duplicated(subset=['Complaint_Number', 'Location_Code'], keep=False)
df1_wids['Duplicates_CN_ST']=df1_wids.duplicated(subset=['Complaint_Number', 'Street'], keep=False)
df1_wids['Duplicates_CN_CD']=df1_wids.duplicated(subset=['Complaint_Number', 'Complaint_Date'], keep=False)
df1_wids['Duplicates_CN_ClD']=df1_wids.duplicated(subset=['Complaint_Number', 'Closed_Date'], keep=False)
df1_wids['Duplicates_CN_IN']=df1_wids.duplicated(subset=['Complaint_Number', 'Investigator_Name'], keep=False)
df1_wids['Duplicates_CN_ACC']=df1_wids.duplicated(subset=['Complaint_Number', 'Accused_Complaint_Category'], keep=False)
df1_wids['Duplicates_CN_AF']=df1_wids.duplicated(subset=['Complaint_Number', 'Accused_Finding'], keep=False)
df1_wids['Duplicates_CN_AFF']=df1_wids.duplicated(subset=['Complaint_Number', 'Accused_Final_Finding'], keep=False)
df1_wids['Duplicates_CN_AD']=df1_wids.duplicated(subset=['Complaint_Number', 'Accused_Discipline'], keep=False)
df1_wids['Duplicates_CN_PWN']=df1_wids.duplicated(subset=['Complaint_Number', 'PO_Witness_Name'], keep=False)
df1_wids['Duplicates_CN_VR']=df1_wids.duplicated(subset=['Complaint_Number', 'Victim_Race_Desc'], keep=False)
df1_wids['Duplicates_CN_CR']=df1_wids.duplicated(subset=['Complaint_Number', 'Complainant_Race_Desc'], keep=False)
print('Number of records w/ matching Complaint Numbers........................',len(df1_wids[df1_wids.Duplicates_CN==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Incident date......',len(df1_wids[df1_wids.Duplicates_CN_IDT==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Beat...............',len(df1_wids[df1_wids.Duplicates_CN_B==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Location code......',len(df1_wids[df1_wids.Duplicates_CN_LC==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Street.............',len(df1_wids[df1_wids.Duplicates_CN_ST==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Complaint date.....',len(df1_wids[df1_wids.Duplicates_CN_CD==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Closed date........',len(df1_wids[df1_wids.Duplicates_CN_ClD==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Investigator.......',len(df1_wids[df1_wids.Duplicates_CN_IN==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Complaint category.',len(df1_wids[df1_wids.Duplicates_CN_ACC==True]),' records')
print('Number of records w/ matching Complaint Numbers AND findings...........',len(df1_wids[df1_wids.Duplicates_CN_AF==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Final findings.....',len(df1_wids[df1_wids.Duplicates_CN_AFF==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Discipline.........',len(df1_wids[df1_wids.Duplicates_CN_AD==True]),' records')
print('Number of records w/ matching Complaint Numbers AND PO witness name....',len(df1_wids[df1_wids.Duplicates_CN_PWN==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Victim race........',len(df1_wids[df1_wids.Duplicates_CN_VR==True]),' records')
print('Number of records w/ matching Complaint Numbers AND Complainant race........',len(df1_wids[df1_wids.Duplicates_CN_CR==True]),' records')

Number of records w/ matching Complaint Numbers........................ 99738  records
Number of records w/ matching Complaint Numbers AND Incident date...... 99738  records
Number of records w/ matching Complaint Numbers AND Beat............... 99738  records
Number of records w/ matching Complaint Numbers AND Location code...... 99738  records
Number of records w/ matching Complaint Numbers AND Street............. 99738  records
Number of records w/ matching Complaint Numbers AND Complaint date..... 99738  records
Number of records w/ matching Complaint Numbers AND Closed date........ 99738  records
Number of records w/ matching Complaint Numbers AND Investigator....... 99731  records
Number of records w/ matching Complaint Numbers AND Complaint category. 97706  records
Number of records w/ matching Complaint Numbers AND findings........... 96918  records
Number of records w/ matching Complaint Numbers AND Final findings..... 96628  records
Number of records w/ matching Complaint Num

What the above tells us is that as far as complaint characteristics go (date,time,location), each complaint number has a single set of characteristics regardless of how many officers are involved. For details of the investigation into each complaints (i.e. investigator, type of complaint, findings), the matches are not perfect, but not that far from it. 